## Config

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *

## GDAX

In [ ]:
# https://github.com/danpaquin/gdax-python
# https://docs.gdax.com

# Use the sandbox API (requires a different set of API access credentials)
gdax_client = gdax.AuthenticatedClient(cfg.GDAX_API_KEY, cfg.GDAX_API_SECRET_KEY, 
                                       cfg.GDAX_PASSPHRASE, api_url=cfg.GDAX_ENDPOINT)

### Historical Prices

In [ ]:
now = datetime.datetime.now()
prior = now - datetime.timedelta(minutes=30)
gdax_client.get_product_historic_rates('ETH-USD', start=prior.isoformat(), end=now.isoformat(), granularity=1800)
gdax_client.get_product_historic_rates('BTC-USD', start=prior.isoformat(), end=now.isoformat(), granularity=1800)
gdax_client.get_product_historic_rates('LTC-USD', start=prior.isoformat(), end=now.isoformat(), granularity=1800)

### Exchange Metadata

In [ ]:
gdax_client.get_products()
gdax_client.get_currencies()
gdax_client.get_time()

### Current Prices

In [ ]:
# Get the order book at the default level.
gdax_client.get_product_order_book('BTC-USD')
# Get the order book at a specific level.
gdax_client.get_product_order_book('BTC-USD', level=1)

In [ ]:
# Get the product ticker for a specific product.
gdax_client.get_product_ticker(product_id='ETH-USD')

In [ ]:
# Get the product trades for a specific product.
gdax_client.get_product_trades(product_id='ETH-USD')

In [ ]:
gdax_client.get_product_24hr_stats('ETH-USD')

### Accounts

In [ ]:
## Accounts - DANGER!
gdax_client.get_accounts()
# gdax_client.get_account(cfg.GDAX_BTC_ACCOUNT)
# gdax_client.get_account(cfg.GDAX_USD_ACCOUNT)
# gdax_client.get_account_history(cfg.GDAX_BTC_ACCOUNT)
# gdax_client.get_account_holds(cfg.GDAX_BTC_ACCOUNT)

### Buy/Sell

In [ ]:
# Buy 0.01 BTC @ 100 USD
gdax_client.buy(price='100.00', #USD
               size='0.01', #BTC
               product_id=c.BTC_USD)

In [ ]:
# Sell 0.01 BTC @ 200 USD
gdax_client.sell(price='200.00', #USD
                size='0.01', #BTC
                product_id=c.BTC_USD)

### Orders

In [ ]:
gdax_client.get_orders(product_id=c.BTC_USD)

In [ ]:
order_id = '9a7ffa16-017a-4ec4-9c8f-31b6e92a7878'
order = gdax_client.get_order(order_id)
order

In [ ]:
gdax_client.get_fills()
gdax_client.get_fills(product_id=c.ETH_USD)
gdax_client.get_fills(order_id=order_id)

In [ ]:
gdax_client.cancel_order(order_id)

### Deposit / Withdraw

In [ ]:
# https://docs.gdax.com/#depositwithdraw

# Deposit from Coinbase into GDAX
deposit = {
    'amount': '25.00', # Currency determined by account specified
    'coinbase_account_id': ''
}
gdax_client.deposit(deposit)

In [ ]:
# Withdraw from GDAX into Coinbase
withdraw = {
    'amount': '1.00', # Currency determined by account specified
    'coinbase_account_id': ''
}
gdax_client.withdraw(withdraw)

### Streaming

In [ ]:
class myWebsocketClient(gdax.WebsocketClient):
    def on_open(self):
        self.url = cfg.GDAX_WEBSOCKET
        self.products = [c.BTC_USD]
        self.message_count = 0
        print("Lets count the messages!")
    
    def on_message(self, msg):
        self.message_count += 1
        if 'price' in msg and 'type' in msg:
            print ("Message type:", msg["type"],
                   "\t@ {:.3f}".format(float(msg["price"])))
    
    def on_close(self):
        print("-- Goodbye! --")

        
wsClient = myWebsocketClient()
wsClient.start()
print(wsClient.url, wsClient.products)
while (wsClient.message_count < 500):
    print ("\nmessage_count =", "{} \n".format(wsClient.message_count))
    time.sleep(1)
wsClient.close()

In [ ]:
wsClient.close()

## Twitter

In [ ]:
# Pass